In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
#loading parts of the available data
#these files are not included here, canbe downloaded from: https://lehd.ces.census.gov/data/lodes/LODES7/tn/od/
df1 = pd.read_csv('tn_od_main_JT00_2019.csv').rename(columns = {'S000':'total_jobs'})
df2 = pd.read_csv('tn_od_main_JT01_2019.csv').rename(columns = {'S000':'total_jobs'})
df3 = pd.read_csv('tn_od_main_JT02_2019.csv').rename(columns = {'S000':'total_jobs'})
df4 = pd.read_csv('tn_od_main_JT03_2019.csv').rename(columns = {'S000':'total_jobs'})
df5 = pd.read_csv('tn_od_main_JT04_2019.csv').rename(columns = {'S000':'total_jobs'})
df6 = pd.read_csv('tn_od_main_JT05_2019.csv').rename(columns = {'S000':'total_jobs'})

In [3]:
#appending all the sources
df = df1.append(df2).append(df3).append(df4).append(df5).append(df6)

In [4]:
#filtering out duplicates 
tn_lodes = df.drop_duplicates()
tn_lodes.h_geocode = tn_lodes.h_geocode.astype(str)
tn_lodes.w_geocode = tn_lodes.w_geocode.astype(str)

c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [5]:
#read Hamilton county blocks (too large to store in github)
# can be downloaded from : https://vanderbilt365-my.sharepoint.com/:f:/g/personal/rishav_sen_vanderbilt_edu/EuB8qV7yx3ZDoxpXq232E1cBJ1Q3Qlzr1cQOvP3UKWqmHw?e=cc1z5h
blocks  = gpd.read_file('../data/Tennessee Census Block/tl_2020_47_tabblock20.shp')
blocks = blocks[blocks.COUNTYFP20 == '065'][['GEOID20', 'geometry']]
blocks.GEOID20 = blocks.GEOID20.astype(str)

In [6]:
# filtering TN LODES data for blocks only in Hamilton county
ham_lodes = pd.merge(tn_lodes, blocks, left_on='h_geocode', right_on='GEOID20', how='inner').merge(blocks, left_on='w_geocode', right_on='GEOID20', how='inner').sort_values('total_jobs', ascending=False).reset_index(drop=True)

In [15]:
# it is stored at the block level (smaller area than CBG)

ham_lodes.to_csv('hamilton_lodes_2019.csv', index=False)

In [39]:
#additional 
pd.merge(tn_lodes, blocks, left_on='h_geocode', right_on='GEOID20', how='inner').groupby('h_geocode').sum().merge(blocks, left_on='h_geocode', right_on='GEOID20', how='inner').sort_values('total_jobs', ascending=False).reset_index().to_csv('homes_blocks.csv')
pd.merge(tn_lodes, blocks, left_on='w_geocode', right_on='GEOID20', how='inner').groupby('w_geocode').sum().merge(blocks, left_on='w_geocode', right_on='GEOID20', how='inner').sort_values('total_jobs', ascending=False).reset_index().to_csv('work_blocks.csv')